# Investment Robo-advisor

## 1. Imports

In [ ]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import statistics
from tqdm import tqdm
from collections import defaultdict, Counter
import math
from itertools import combinations
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from tensorflow.keras.models import Sequential # type: ignore
from tensorflow.keras.layers import LSTM, Dense, Dropout # type: ignore

from fetchData import fetch_raw_data_yf, getSNP500, fetch_raw_data_yf_all, getNasdaq_comp
from LearningRBA import MLRBA_V1, MLRBA_V2
from MonteCarloRBA import MonteCarloRBA
from PortfolioFunction import maximize_sharpe, create_correlation_matrix, get_sharpe_ratio, get_matrices, maximize_sharpe_SLSQP
from LearningRBA import find_best_asset_to_remove, find_asset_to_add


## 2. Fetch Data

### Get all Nasdaq Stocks

In [ ]:
assets= [
    "AAPL",  # Apple Inc.
    "MSFT",  # Microsoft Corporation
    "AMZN",  # Amazon.com Inc.
    "GOOGL", # Alphabet Inc. (Google) Class A
    "GOOG",  # Alphabet Inc. (Google) Class C
    "META",    # Meta Platforms Inc (formerly Facebook)
    "TSLA",  # Tesla Inc
    "UA", # Berkshire Hathaway Inc. Class B
    "JPM",   # JPMorgan Chase & Co.
    "V",     # Visa Inc.
    "JNJ",   # Johnson & Johnson
    "WMT",   # Walmart Inc.
    "PG",    # Procter & Gamble Co.
    "UNH",   # UnitedHealth Group Inc.
    "MA",    # Mastercard Inc.
    "NVDA",  # NVIDIA Corporation
    "HD",    # Home Depot Inc.
    "BAC",   # Bank of America Corp
    "DIS",   # Walt Disney Co
    "PYPL",  # PayPal Holdings
    "VZ",    # Verizon Communications Inc.
    "ADBE",  # Adobe Inc.
    "CMCSA", # Comcast Corporation
    "NFLX",  # Netflix Inc.
    "KO",    # Coca-Cola Co
    "NKE",   # NIKE Inc.
    "PFE",   # Pfizer Inc.
    "MRK",   # Merck & Co., Inc.
    "PEP",   # PepsiCo, Inc.
    "T",     # AT&T Inc.
    "ABT",   # Abbott Laboratories
    "CRM",   # Salesforce.com Inc.
    "ORCL",  # Oracle Corporation
    "ABBV",  # AbbVie Inc.
    "CSCO",  # Cisco Systems, Inc.
    "INTC",  # Intel Corporation
    "TMO",   # Thermo Fisher Scientific Inc.
    "XOM",   # Exxon Mobil Corporation
    "ACN",   # Accenture plc
    "LLY",   # Eli Lilly and Company
    "COST",  # Costco Wholesale Corporation
    "MCD",   # McDonald's Corp
    "DHR",   # Danaher Corporation
    "MDT",   # Medtronic plc
    "NEE",   # NextEra Energy, Inc.
    "BMY",   # Bristol-Myers Squibb Company
    "QCOM",  # Qualcomm Inc
    "CVX",   # Chevron Corporation
    "WFC",   # Wells Fargo & Co
    "LMT",    # Lockheed Martin Corporation
    "GS",   # Goldman Sachs Group, Inc.
    "MS",   # Morgan Stanley
    "IBM",  # International Business Machines Corporation
    "GE",   # General Electric Company
    "F",    # Ford Motor Company
    "GM",   # General Motors Company
    "UBER", # Uber Technologies, Inc.
    "LYFT", # Lyft, Inc.
    "SNAP", # Snap Inc.
    "TWTR", # Twitter, Inc.
    "SPOT", # Spotify Technology S.A.
    "AMD",  # Advanced Micro Devices, Inc.
    "TXN",  # Texas Instruments Incorporated
    "BABA", # Alibaba Group Holding Limited
    "SAP",  # SAP SE
    "HON",  # Honeywell International Inc.
    "BA",   # Boeing Company
    "RTX",  # Raytheon Technologies Corporation
    "CAT",  # Caterpillar Inc.
    "DE",   # Deere & Company
    "MMM",  # 3M Company
    "DUK",  # Duke Energy Corporation
    "SO",   # Southern Company
    "EXC",  # Exelon Corporation
    "NEE",  # NextEra Energy, Inc.
    "AEP",  # American Electric Power Company, Inc.
    "SRE",  # Sempra Energy
    "ETN",  # Eaton Corporation plc
    "EMR",  # Emerson Electric Co.
    "SYY",  # Sysco Corporation
    "KR",   # Kroger Co.
    "GIS",  # General Mills, Inc.
    "K",    # Kellogg Company
    "CPB",  # Campbell Soup Company
    "MO",   # Altria Group, Inc.
    "PM",   # Philip Morris International Inc.
    "BTI",  # British American Tobacco plc
    "RDY",  # Dr. Reddy's Laboratories Ltd.
    "GILD", # Gilead Sciences, Inc.
    "BIIB", # Biogen Inc.
    "CELG", # Celgene Corporation
    "AMGN", # Amgen Inc.
    "SYK",  # Stryker Corporation
    "BSX",  # Boston Scientific Corporation
    "ISRG", # Intuitive Surgical, Inc.
    "ZBH",  # Zimmer Biomet Holdings, Inc.
    "EW",   # Edwards Lifesciences Corporation
    "RMD",  # ResMed Inc.
    "VRTX", # Vertex Pharmaceuticals Incorporated
    "REGN",  # Regeneron Pharmaceuticals, Inc.
]

assets = getSNP500()

In [ ]:
start_date = pd.Timestamp("2024-01-01")
end_date = pd.Timestamp("2025-01-01")

In [ ]:
raw_data, asset_errors, max_combination= fetch_raw_data_yf(assets, start_date, end_date)

## 3. Mean, Volatility and Covariance

In [ ]:
names, annualized_returns, unweighted_annaulized_returns, weighted_returns_matrix, normal_returns_matrix, cov, correlation_matrix = get_matrices(raw_data)

volatility = np.sqrt(np.diag(cov))
risk_free_rate=0
sharpe_ratios = (annualized_returns - risk_free_rate) / volatility

In [ ]:
hover_texts = [
    f"<br>Symbol: {ticker} <br>Volatility: {vol:.3f} <br>Returns: {ret:.3%} <br>Sharpe Ratio: {sr:.3f}"
    for ticker, vol, ret, sr in zip(names, volatility, annualized_returns, sharpe_ratios)
]

fig = go.Figure(data=go.Scatter(
    x=volatility,
    y=annualized_returns,
    mode='markers',
    hoverinfo='text',
    hovertext=hover_texts,
    marker=dict(color=sharpe_ratios, colorscale = 'RdBu', size=6, line=dict(width=1), colorbar=dict(title="Sharpe<br>Ratio")
    )
))

fig.update_layout(
    title='Annual Performance of Individual Assets',
    xaxis_title='Volatility (Standard Deviation)',
    yaxis_title='Annualized Returns',
    width = 1920,
    height = 1080,
    font=dict(
        family="Cambria",
        size=18,
    )
)

fig.show()
#fig.write_html("PerformanceofIndividualAssets.html")
#fig.write_image("PerformanceofIndividualAssets.png", format='png', width=1920, height=1080)

### Demonstrating SLSQP

In [ ]:
rand_assets = np.random.choice(list(names), 5, replace=False)

selected_returns = annualized_returns.loc[rand_assets].values
selected_covariances = cov.loc[rand_assets, rand_assets].values

optimized_weights, weights_history, sharpe_history = maximize_sharpe_SLSQP(selected_returns, selected_covariances)

In [ ]:
fig = make_subplots(rows=1, cols=2, subplot_titles=("Portfolio Weights Over Iterations", "Sharpe Ratio Over Iterations"))

for i in range(len(selected_returns)):
    fig.add_trace(go.Scatter(x=list(range(len(weights_history))), 
                             y=[h[i] for h in weights_history], 
                             mode='lines+markers', 
                             name=f'Asset {i+1} Weight'), 
                  row=1, col=1)

fig.add_trace(go.Scatter(x=list(range(len(sharpe_history))), 
                         y=sharpe_history, 
                         mode='lines+markers', 
                         name='Sharpe Ratio'), 
              row=1, col=2)

fig.update_layout(title_text='Portfolio Optimization Analysis',
                  xaxis_title='Iteration',
                  yaxis_title='Weight',
                  legend_title='Assets',
                    font=dict(
                        family="Cambria",
                        size=18,
                    )
)

# Update xaxis and yaxis properties for Sharpe Ratio subplot
fig.update_xaxes(title_text="Iteration", row=1, col=2)
fig.update_yaxes(title_text="Sharpe Ratio", row=1, col=2)

fig.show()

#fig.write_html("SLSQPDemo.html")
#fig.write_image("SLSQPDemo.png", format='png', width=1920, height=1080)

## 4.0 Monte Carlo Method

In [ ]:
all_portfolios, dominant_portfolios = MonteCarloRBA(names, cov, annualized_returns, 10000, min_assets=8, max_assets=8)

In [ ]:
iterations = [portfolio['iteration'] for portfolio in dominant_portfolios]
counts = list(range(1, len(dominant_portfolios) + 1))

fig = go.Figure()

fig.add_trace(go.Scatter(x=iterations, y=counts,
                        mode='lines',
                        name='Dominant Portfolios',
                        line=dict(shape='spline')
))  

fig.update_layout(
    title='Growth of Dominant Portfolios Over Iterations',
    xaxis_title='Portfolios Generated',
    yaxis_title='Number of Dominant Portfolios Found',
    height=1080,
    width=1920,
    font=dict(
        family="Cambria",
        size=18,
    )
)

fig.show()
#fig.write_html("FrequencyOfDom.html")
#fig.write_image("FrequencyOfDom.png", format='png', width=1920, height=1080)

In [ ]:
fig1 = go.Figure()

fig1.add_trace(go.Scatter(
    x=[np.sqrt(p["variance"]) for p in all_portfolios],
    y=[p["return"] for p in all_portfolios],
    mode='markers',
    marker=dict(
        color=[p["sharpe"] for p in all_portfolios],
        showscale=True,
        size=7,
        line=dict(width=1),
        colorscale="RdBu",
        colorbar=dict(title="Sharpe<br>Ratio")
    ),
    hoverinfo='text',
    text=[
        f"Return: {p['return']:.3%}<br>Volatility: {np.sqrt(p['variance']):.3f}<br>" +
        f"Sharpe Ratio: {p['return'] / (np.sqrt(p['variance'])):.3f}<br>" +
        "<br>".join([f"{p['tickers'][i]}: Weight={p['weights'][i]:.3f}" for i in range(len(p['tickers']))])
        for p in all_portfolios
    ]
))

fig1.update_layout(
    xaxis=dict(title='Volatility (Standard Deviation)'),
    yaxis=dict(title='Annualised Returns'),
    title='Monte Carlo Randomly Generated Portfolios',
    height=1080,
    width=1920,
    font=dict(
        family="Cambria",
        size=18,
    )
)
fig1.show()
#fig1.write_html("MonteCarlo.html")
#fig1.write_image("MonteCarlo.png", format='png', width=1920, height=1080)

In [ ]:
fig2 = go.Figure()

fig2.add_trace(go.Scatter(
    x=[np.sqrt(p["variance"]) for p in dominant_portfolios],  # Convert variance to volatility
    y=[p["return"] for p in dominant_portfolios],
    mode='markers',
    marker=dict(
        size=7,
        line=dict(width=1),
        #showscale=True,
        #color=[p["return"] / (np.sqrt(p["variance"])) for p in dominant_portfolios],  # Sharpe Ratio
        #colorscale="RdBu",
        #colorbar=dict(title="Sharpe<br>Ratio")
    ),
    hoverinfo='text',
    text=[
        f"Return: {p['return']:.3%}<br>Volatility: {np.sqrt(p['variance']):.3f}<br>" +
        f"Sharpe Ratio: {p['sharpe']:.3f}<br>" +
        "<br>".join([f"{p['tickers'][i]}: Weight={p['weights'][i]:.3f}" for i in range(len(p['tickers']))])
        for p in dominant_portfolios
    ],
    name="Monte Carlo Portfolios"
))

fig2.add_trace(go.Scatter(
    x=volatility,
    y=annualized_returns,
    mode='markers',
    hoverinfo='text',
    hovertext=[
        f"{name} <br>Volatility: {vol:.3f} <br>Returns: {ret:.3%} <br>Sharpe Ratio: {sr:.3f}"
        for name, vol, ret, sr in zip(names, volatility, annualized_returns, sharpe_ratios)
    ],
    marker=dict(
        color='green',
        size=5,
        line=dict(width=1)
    ),
    name="Individual Assets"
))

fig2.update_layout(
    title='Monte Carlo Portfolios with Individual Assets',
    xaxis_title='Volatility (Standard Deviation)',
    yaxis_title='Annualized Return',
    legend=dict(y=5),
    height=1080,
    width=1920,
    font=dict(
        family="Cambria",
        size=18,
    )
)

fig2.show()
#fig2.write_html("MCMarkowitzBullet.html")
#fig2.write_image("MCMarkowitzBullet.png", format='png', width=1920, height=1080)

## 5.0 Machine Learning Method

### 5.1 Optimization Function Only

In [ ]:
base_portfolio, best_portfolio, good_portfolios, total_portfolios, best_iteration = MLRBA_V1(names, cov, annualized_returns)
base_portfolio, best_portfolio, len(good_portfolios), len(total_portfolios), best_iteration

In [ ]:
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=[p["variance"]**0.5 for p in good_portfolios],  # Convert variance to volatility
    y=[p["return"] for p in good_portfolios],
    mode='markers',
    marker=dict(
        color=[p["sharpe"] for p in good_portfolios],  # Sharpe Ratio
        showscale=True,
        size=7,
        line=dict(width=1),
        colorscale="RdBu",
        colorbar=dict(title="Sharpe<br>Ratio")
    ),
    hoverinfo='text',
    text=[
        f"Return: {p['return']:.3%}<br>Volatility: {p['variance']**0.5:.3f}<br>" +
        f"Sharpe Ratio: {p['return'] / (p['variance']**0.5):.3f}<br>" +
        "<br>".join([f"{p['tickers'][i]}: Weight={p['weights'][i]:.3f}" for i in range(len(p['tickers']))])
        for p in good_portfolios
    ],
    name="Portfolios"
))

fig.update_layout(
    title='Convergence Strategy Generarted Portfolios',
    xaxis_title='Volatility (Standard Deviation)',
    yaxis_title='Annualized Return',
    legend=dict(y=5),
    width=1920,
    height=1080,
    font=dict(
        family="Cambria",
        size=18,
    )
)

fig.show()
#fig.write_html("ConvergenceRBA.html")
#fig.write_image("ConvergenceRBA.png", format='png', width=1920, height=1080)

fig.add_trace(go.Scatter(
    x=volatility,
    y=annualized_returns,
    mode='markers',
    hoverinfo='text',
    hovertext=[
        f"{name} <br>Volatility: {vol:.3f} <br>Returns: {ret:.3%} <br>Sharpe Ratio: {sr:.3f}"
        for name, vol, ret, sr in zip(names, volatility, annualized_returns, sharpe_ratios)
    ],
    marker=dict(
        color='green',
        size=5,
        line=dict(width=1)
    ),
    name="Individual Assets"
))

fig.update_layout(
    title='Convergence Strategy Generarted Portfolios with Individual Assets',
    xaxis_title='Volatility (Standard Deviation)',
    yaxis_title='Annualized Return',
    legend=dict(y=5),
    width=1920,
    height=1080,
    font=dict(
        family="Cambria",
        size=18,
    )
)


fig.show()

#fig.write_html("ConvergenceRBA+Asset.html")
#fig.write_image("ConvergenceRBA+Asset.png", format='png', width=1920, height=1080)


In [ ]:
sharpe_ratios = [portfolio['sharpe'] for portfolio in total_portfolios]

fig = go.Figure(data=go.Scatter(x=list(range(len(sharpe_ratios))), y=sharpe_ratios, mode='lines+markers'))
fig.update_layout(title='Sharpe Ratio Over Iterations',
                xaxis_title='Iteration',
                yaxis_title='Sharpe Ratio',
                width=1920,
                height=1080,
                font=dict(
                    family="Cambria",
                    size=18,
                )
)
fig.show()

#### Comparing MLRBA_V1 with Monte Carlo

In [ ]:
figC = go.Figure(fig2)
figC.add_trace(go.Scatter(
    x=[p["variance"]**0.5 for p in good_portfolios],  # Convert variance to volatility
    y=[p["return"] for p in good_portfolios],
    mode='markers',
    marker=dict(
        size=7,
        line=dict(width=1),
        color="Red",
    ),
    hoverinfo='text',
    text=[
        f"Return: {p['return']:.3%}<br>Volatility: {p['variance']**0.5:.3f}<br>" +
        f"Sharpe Ratio: {p['return'] / (p['variance']**0.5):.3f}<br>" +
        "<br>".join([f"{p['tickers'][i]}: Weight={p['weights'][i]:.3f}" for i in range(len(p['tickers']))])
        for p in good_portfolios
    ],
    name="Convergence Portfolios"
))

figC.update_layout(
    title='Convergence vs Monte Carlo vs Individual Assets',
    legend=dict(x=0.85, y=0.95),
    width=1920,
    height=1080,
    font=dict(
        family="Cambria",
        size=18,
    )
)

figC.show()
#figC.write_html("ComparisonOfConvergence+MC.html")
#figC.write_image("ComparisonOfConvergence+MC.png", format='png', width=1920, height=1080)

### 5.2 Reinforcement Weight Training

In [ ]:
base_portfolio, best_portfolio, good_portfolios, total_portfolios, best_iteration = MLRBA_V2(names, cov, annualized_returns, correlation_matrix)
base_portfolio, best_portfolio, len(good_portfolios), len(total_portfolios), best_iteration

In [ ]:
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=[p["variance"]**0.5 for p in good_portfolios],  # Convert variance to volatility
    y=[p["return"] for p in good_portfolios],
    mode='markers',
    marker=dict(
        color=[p["sharpe"] for p in good_portfolios],  # Sharpe Ratio
        showscale=True,
        size=7,
        line=dict(width=1),
        colorscale="RdBu",
        colorbar=dict(title="Sharpe<br>Ratio")
    ),
    hoverinfo='text',
    text=[
        f"Return: {p['return']:.3%}<br>Volatility: {p['variance']**0.5:.3f}<br>" +
        f"Sharpe Ratio: {p['return'] / (p['variance']**0.5):.3f}<br>" +
        "<br>".join([f"{p['tickers'][i]}: Weight={p['weights'][i]:.3f}" for i in range(len(p['tickers']))])
        for p in good_portfolios
    ],
    name="Portfolios"
))

fig.update_layout(
    title='Learning Convergence Portfolios',
    xaxis_title='Volatility (Standard Deviation)',
    yaxis_title='Annualized Return',
    legend=dict(y=5),
    width=1920,
    height=1080,
    font=dict(
        family="Cambria",
        size=18,
    )
)

fig.show()
#fig.write_html("LearningConvergenceRBA.html")
#fig.write_image("LearningConvergenceRBA.png", format='png', width=1920, height=1080)

fig.add_trace(go.Scatter(
    x=volatility,
    y=annualized_returns,
    mode='markers',
    hoverinfo='text',
    hovertext=[
        f"{name} <br>Volatility: {vol:.3f} <br>Returns: {ret:.3%} <br>Sharpe Ratio: {sr:.3f}"
        for name, vol, ret, sr in zip(names, volatility, annualized_returns, sharpe_ratios)
    ],
    marker=dict(
        color='green',
        size=5,
        line=dict(width=1)
    ),
    name="Individual Assets"
))

fig.update_layout(
    title='Learning Convergence Portfolios with Individual Assets',
    xaxis_title='Volatility (Standard Deviation)',
    yaxis_title='Annualized Return',
    legend=dict(y=5),
    width=1920,
    height=1080,
    font=dict(
        family="Cambria",
        size=18,
    )
)


fig.show()
#fig.write_html("LearningConvergenceRBA+Asset.html")
#fig.write_image("LearningConvergenceRBA+Asset.png", format='png', width=1920, height=1080)


In [ ]:
sharpe_ratios = [portfolio['sharpe'] for portfolio in total_portfolios]

fig = go.Figure(data=go.Scatter(x=list(range(len(sharpe_ratios))), y=sharpe_ratios, mode='lines+markers'))
fig.update_layout(title='Sharpe Ratio Over Iterations',
                  xaxis_title='Iteration',
                  yaxis_title='Sharpe Ratio',
                  )
fig.show()

#### Comparing MLRBA_V2 with Monte Carlo

In [ ]:
figC = go.Figure(fig2)
figC.add_trace(go.Scatter(
    x=[p["variance"]**0.5 for p in good_portfolios],  # Convert variance to volatility
    y=[p["return"] for p in good_portfolios],
    mode='markers',
    marker=dict(
        size=7,
        line=dict(width=1),
        color="Red",
    ),
    hoverinfo='text',
    text=[
        f"Return: {p['return']:.3%}<br>Volatility: {p['variance']**0.5:.3f}<br>" +
        f"Sharpe Ratio: {p['return'] / (p['variance']**0.5):.3f}<br>" +
        "<br>".join([f"{p['tickers'][i]}: Weight={p['weights'][i]:.3f}" for i in range(len(p['tickers']))])
        for p in good_portfolios
    ],
    name="Learning Convergence Portfolios"
))

figC.update_layout(
    title='Learning Convergence vs Monte Carlo vs Individual Assets',
    legend=dict(x=0.8, y=0.95),
    width=1920,
    height=1080,
    font=dict(
        family="Cambria",
        size=18,
    )
)

figC.show()
#figC.write_html("ComparisonOfLearningConvergence+MC.html")
#figC.write_image("ComparisonOfLearningConvergence+MC.png", format='png', width=1920, height=1080)

### Comparing V1 and V2

In [ ]:
def run_MLRBA_multiple_times(names, cov, annualized_returns, num_runs, num_assets=8):

    def generate_rand_port(tickers, num_assets, num_runs):
        rand_port = []
        for _ in range (num_runs):
            base_portfolio = np.random.choice(list(tickers), num_assets, replace=False)
            rand_port.append(base_portfolio)

        return rand_port
    
    random_portfolios = generate_rand_port(names, num_assets=num_assets, num_runs=num_runs)

    total_good_portfolios_length_v1 = 0
    best_portfolios_v1 = []
    best_iterations_v1 = []
    
    total_good_portfolios_length_v2 = 0
    best_portfolios_v2 = []
    best_iterations_v2 = []
    
    for portfolio in random_portfolios:
        base_portfolio_v1, best_portfolio_v1, good_portfolios_v1, _, best_iteration_v1 = MLRBA_V1(names, cov, annualized_returns, base_portfolio=portfolio)
        total_good_portfolios_length_v1 += len(good_portfolios_v1)
        best_portfolios_v1.append(best_portfolio_v1)
        best_iterations_v1.append(best_iteration_v1)
        
        base_portfolio_v2, best_portfolio_v2, good_portfolios_v2, _, best_iteration_v2 = MLRBA_V2(names, cov, annualized_returns, correlation_matrix, base_portfolio=portfolio)
        total_good_portfolios_length_v2 += len(good_portfolios_v2)
        best_portfolios_v2.append(best_portfolio_v2)
        best_iterations_v2.append(best_iteration_v2)

        print(base_portfolio_v1['tickers'] == base_portfolio_v2['tickers'])

    average_length_v1 = total_good_portfolios_length_v1 / num_runs
    average_iteration_v1 = statistics.fmean(best_iterations_v1)
    std_dev_iteration_v1 = statistics.stdev(best_iterations_v1) if num_runs > 1 else 0

    average_length_v2 = total_good_portfolios_length_v2 / num_runs
    average_iteration_v2 = statistics.fmean(best_iterations_v2)
    std_dev_iteration_v2 = statistics.stdev(best_iterations_v2) if num_runs > 1 else 0

    
    results = {
        'v1': (base_portfolio_v1, average_length_v1, best_portfolios_v1, average_iteration_v1, std_dev_iteration_v1, best_iterations_v1),
        'v2': (base_portfolio_v2, average_length_v2, best_portfolios_v2, average_iteration_v2, std_dev_iteration_v2, best_iterations_v2)
    }
    
    return results

num_runs = 20
results = run_MLRBA_multiple_times(names, cov, annualized_returns, num_runs)

_, _, best_portfolios_v1, average_iteration_v1, std_dev_v1, best_iterations_v1 = results['v1']
_, _, best_portfolios_v2, average_iteration_v2, std_dev_v2, best_iterations_v2 = results['v2']

In [ ]:
from plotly.subplots import make_subplots

avg_sharpe_v1 = np.mean([portfolio['sharpe'] for portfolio in best_portfolios_v1])
avg_sharpe_v2 = np.mean([portfolio['sharpe'] for portfolio in best_portfolios_v2])

fig = make_subplots(
    rows=1, cols=2,
    subplot_titles=[
        "Average Iterations to Find Best Portfolio",
        "Average Highest Sharpe Ratio"
    ]
)

fig.add_trace(go.Bar(
    x=['Standard Convergence', 'Learning Convergergence'],
    y=[average_iteration_v1, average_iteration_v2],
    name='Iterations',
    error_y=dict(type='data', array=[std_dev_v1, std_dev_v2], visible=True),
    width=0.4
), row=1, col=1)

fig.add_trace(go.Bar(
    x=['Standard Convergence', 'Learning Convergergence'],
    y=[avg_sharpe_v1, avg_sharpe_v2],
    name='Sharpe Ratio',
    width=0.4
), row=1, col=2)

# Axis titles
fig.update_xaxes(title_text='Method Version', row=1, col=1)
fig.update_yaxes(title_text='Average Iterations', row=1, col=1)

fig.update_xaxes(title_text='Method Version', row=1, col=2)
fig.update_yaxes(title_text='Average Sharpe Ratio', row=1, col=2)

# Layout and display
fig.update_layout(
    title_text='Standard Convergence vs Learning Convergence',
    showlegend=False,
    font=dict(
        family="Cambria",
        size=18,
    )
)

fig.show()
#fig.write_html("StandardvsLearning.html")
#fig.write_image("StandardvsLearning.png", format='png', width=1920, height=1080)

## 6.0 Portfolio Prediction using LSTM

In [ ]:
class PortfolioPredictor:
    def __init__(self, raw_data_train, raw_data_test, best_portfolio, n_steps=1, epochs=50, batch_size=32):
        self.raw_data_train = raw_data_train
        self.raw_data_test = raw_data_test
        self.best_portfolio = best_portfolio
        self.n_steps = n_steps
        self.epochs = epochs
        self.batch_size = batch_size
        self.model = None
        self.history = None

    def preprocess_data(self):
        best_portfolio_data_train = self.raw_data_train[self.best_portfolio['tickers']]
        best_portfolio_data_test = self.raw_data_test[self.best_portfolio['tickers']]
        weights = np.array(self.best_portfolio['weights'])

        # Use a scaler fitted on a broader dataset so that training/test normalization is consistent
        self.scaler = MinMaxScaler(feature_range=(0, 1))
        # Fit on the entire raw_data (or on a fixed training period) for consistency
        full_data = pd.concat([best_portfolio_data_train, best_portfolio_data_test])
        self.scaler.fit(full_data)
        
        normalized_train_data = self.scaler.transform(best_portfolio_data_train)
        normalized_test_data = self.scaler.transform(best_portfolio_data_test)

        self.weighted_returns_train = np.dot(normalized_train_data, weights)
        self.weighted_returns_test = np.dot(normalized_test_data, weights)

    def create_datasets(self, data):
        X, y = [], []
        for i in range(len(data) - self.n_steps):
            v = data[i:(i + self.n_steps), :]
            X.append(v)
            y.append(data[i + self.n_steps, :])
        return np.array(X), np.array(y)

    def build_model(self):
        self.model = Sequential([
            LSTM(250, activation='tanh', return_sequences=True),
            Dropout(0.2),
            LSTM(50, activation='tanh', return_sequences=False),
            Dropout(0.2),
            Dense(1),
        ])

        def tf_weighted_mse(y_true, y_pred, power=3):
            n = tf.shape(y_true)[0]
            normalized_index = tf.cond(
                tf.equal(n, 1),
                lambda: tf.ones([n], dtype=tf.float32),
                lambda: tf.cast(tf.range(n), tf.float32) / tf.cast(n - 1, tf.float32)
            )
            weights = tf.pow(normalized_index, power)
            weights += 1e-6
            weights /= tf.reduce_sum(weights)
            
            squared_errors = tf.square(y_true - y_pred)
            weighted_squared_errors = weights * squared_errors
            return tf.reduce_mean(weighted_squared_errors)

        self.model.compile(optimizer='adam', loss=tf_weighted_mse)

    def train_model(self):
        self.X_train_weighted, self.y_train_weighted = self.create_datasets(self.weighted_returns_train.reshape(-1, 1))
        self.history = self.model.fit(self.X_train_weighted, self.y_train_weighted, epochs=self.epochs, batch_size=self.batch_size, validation_split=0.001, shuffle=False, verbose=0)

    def predict(self):
        X_test_weighted, y_test_weighted = self.create_datasets(self.weighted_returns_test.reshape(-1, 1))
        
        self.predictions = self.model.predict(X_test_weighted)
        self.y_test_weighted = y_test_weighted
        
        return self.predictions

    def compute_cumulative_returns(self, data, baseline):
        data_series = pd.Series(data.flatten())
        cumulative_returns = data_series / data_series.iloc[0] * baseline
        return cumulative_returns

    def plot_loss(self):
        fig = go.Figure()
        fig.add_trace(go.Scatter(x=np.arange(1, len(self.history.history['loss'])+1), y=self.history.history['loss'], mode='lines', name='Training Loss'))
        fig.add_trace(go.Scatter(x=np.arange(1, len(self.history.history['val_loss'])+1), y=self.history.history['val_loss'], mode='lines', name='Validation Loss'))
        fig.update_layout(title='Training and Validation Loss Over Epochs',
                            xaxis_title='Epoch',
                            yaxis_title='Loss',
                            legend_title='Type of Loss',
                            width=1920,
                            height=1080,
                            font=dict(
                                family="Cambria",
                                size=18,
                            ))
        fig.show()
        
    def plot_predictions(self):
        normalized_train = self.compute_cumulative_returns(self.y_train_weighted, 100)
        training_end_value = normalized_train.iloc[-1]
        normalized_test = self.compute_cumulative_returns(self.y_test_weighted, training_end_value)
        normalized_predicted = self.compute_cumulative_returns(self.predictions, training_end_value)

        fig = go.Figure()
        fig.add_trace(go.Scatter(
            x=self.raw_data_train.index,
            y=normalized_train,
            mode='lines',
            name='Actual Training Returns'
        ))
        fig.add_trace(go.Scatter(
            x=self.raw_data_test.index,
            y=normalized_test,
            mode='lines',
            name='Actual Test Returns'
        ))
        fig.add_trace(go.Scatter(
            x=self.raw_data_test.index,
            y=normalized_predicted,
            mode='lines',
            name='Predicted Test Returns'
        ))
        fig.update_layout(
            title='Actual vs Predicted Weighted Portfolio Returns',
            xaxis_title='Date',
            yaxis_title='Cumulative Returns',
            legend_title='Portfolio',
            width=1920,
            height=1080,
            font=dict(
                family="Cambria",
                size=18,
            )
        )
        fig.show()

In [ ]:
investment_length = 90
investment_end_date = end_date + pd.Timedelta(days=investment_length)

raw_data, asset_errors, max_combination= fetch_raw_data_yf(assets, start_date, investment_end_date)
names, annualized_returns, unweighted_annaulized_returns, weighted_returns_matrix, normal_returns_matrix, cov, correlation_matrix = get_matrices(raw_data)

In [ ]:
raw_data_train = raw_data.loc[:end_date]
raw_data_test = raw_data.loc[end_date:]

portfolio_predictor = PortfolioPredictor(raw_data_train, raw_data_test, best_portfolio, n_steps=1, epochs=30)

portfolio_predictor.preprocess_data()
portfolio_predictor.build_model()
portfolio_predictor.train_model()
prediction = portfolio_predictor.predict()   
portfolio_predictor.plot_loss()
portfolio_predictor.plot_predictions()

### Full Algorithm

In [ ]:
def evaluate_portfolios_over_time(raw_data, window_size=5, threshold=0.05, epochs=30, length_of_investment = None):
    investment_start = len(raw_data[:end_date])

    trading_days = len(raw_data[end_date:end_date + pd.Timedelta(days=length_of_investment)] if length_of_investment is not None else raw_data[end_date:])
    trading_days = trading_days - 1 if trading_days % window_size == 0 else trading_days
    num_windows = trading_days // window_size

    previous_best_portfolio = None
    all_good_portfolios = []
    for i in range(num_windows):
        print(f'Predicting Week {i} in {num_windows} Total Weeks')

        loop_raw_data_train = raw_data.iloc[:investment_start + i*window_size]
        loop_raw_data_test = raw_data.iloc[investment_start + i*window_size:]
        
        loop_names, loop_annualized_returns, _, _, _, loop_cov, loop_correlation_matrix = get_matrices(loop_raw_data_train)
        
        _, loop_best_portfolio, loop_good_portfolios, _, _ = MLRBA_V2(loop_names, loop_cov, loop_annualized_returns, loop_correlation_matrix)
        best_sharpe = loop_best_portfolio['sharpe']
        
        close_to_best = []
        if previous_best_portfolio is not None:
            close_to_best.append(previous_best_portfolio)
        close_to_best.append(loop_best_portfolio)
        
        for j in range(len(loop_best_portfolio)):
            difference = abs((best_sharpe - loop_good_portfolios[j]['sharpe']) / best_sharpe)
            if difference < threshold:
                close_to_best.append(loop_good_portfolios[j])

        print(f'Length of close to best is: {len(close_to_best)}')

        sharpe_list = [portfolio['sharpe'] for portfolio in close_to_best]
        print("Sharpe ratios (first is best_sharpe):", sharpe_list)
        
        portfolio_results = {}
        for id, portfolio in enumerate(close_to_best):
            portfolio_predictor = PortfolioPredictor(loop_raw_data_train, loop_raw_data_test, portfolio, n_steps=window_size, epochs=epochs)
            portfolio_predictor.preprocess_data()
            portfolio_predictor.build_model()
            portfolio_predictor.train_model()
            prediction = portfolio_predictor.predict() 

            if len(prediction) >= window_size:
                end_pred = prediction[window_size-1]
            else:
                end_pred = prediction[-1]
            
            percentage_diff = (end_pred - prediction[0]) / prediction[0]
            print(prediction[:min(window_size, len(prediction))], prediction[0], percentage_diff * 100)          
            
            portfolio_results[id] = percentage_diff

        best_id = None

        # Check if all predictions (percentage_diff) are negative
        if max(portfolio_results.values()) < 0:
            print("All percentage differences are negative. Choosing an empty portfolio(not holding anything).")
            predicted_best_portfolio = {}
        else:
            best_id = max(portfolio_results, key=portfolio_results.get)
            predicted_best_portfolio = close_to_best[best_id]
            previous_best_portfolio = predicted_best_portfolio
        
        portfolio_start_date = loop_raw_data_test.index[0]
        portfolio_end_date = loop_raw_data_test.index[window_size-1]
        
        all_good_portfolios.append({
            "portfolio": predicted_best_portfolio,
            "start_date": portfolio_start_date,
            "end_date": portfolio_end_date
        })
        if best_id is not None:
            print(f'Current iteration: {i}, the best portfolio found was portfolio: {best_id}')
        else:
            print(f'Current iteration: {i}, no portfolio selected (empty portfolio chosen).')
    
    return all_good_portfolios

In [ ]:
all_good_portfolios = evaluate_portfolios_over_time(raw_data, window_size=5, threshold=0.5, epochs=20, length_of_investment = None)

In [ ]:
def extract_asset_returns(raw_data, assets, start_date, end_date):
    if not isinstance(raw_data.index, pd.DatetimeIndex):
        raw_data.index = pd.to_datetime(raw_data.index)

    filtered_data = raw_data.loc[start_date:end_date, assets]

    return filtered_data

def chain_portfolio_performance(weekly_series_list, starting_value=100):
    continuous_series = pd.Series()
    current_value = starting_value

    for week_series in weekly_series_list:
        # Normalize the week so that it starts at 1 (or current_value)
        week_normalized = week_series / week_series.iloc[0]
        # Scale the normalized week to start at current_value
        week_scaled = week_normalized * current_value
        # Update the current_value to the last value of this week
        current_value = week_scaled.iloc[-1]
        # Append the week_series to the continuous_series
        continuous_series = pd.concat([continuous_series, week_scaled])
    
    return continuous_series

ML_portfolio = []
portfolio_holdings = []  # To store tickers held at each point
initial_value = 100

for i in range(len(all_good_portfolios)):
    curr_best_portfolio = all_good_portfolios[i]['portfolio']
    p_start_date = all_good_portfolios[i]['start_date']
    p_end_date = all_good_portfolios[i]['end_date']
    
    if not curr_best_portfolio:
        # No portfolio for this period
        if i == 0:
            previous_value = initial_value
        else:
            previous_value = ML_portfolio[-1].iloc[-1]
        window_index = raw_data.loc[p_start_date:p_end_date].index
        portfolio_daily_returns = pd.Series(previous_value, index=window_index)
        held_tickers = []  # Nothing held
    else:
        # Portfolio exists for this period
        best_curr_port_assets = curr_best_portfolio['tickers']
        best_curr_port_assets_test_data = extract_asset_returns(raw_data, best_curr_port_assets, p_start_date, p_end_date)
        curr_best_portfolio_weights = curr_best_portfolio['weights']
        weighted_returns = best_curr_port_assets_test_data.mul(curr_best_portfolio_weights, axis='columns')
        portfolio_daily_returns = weighted_returns.sum(axis=1)
        held_tickers = best_curr_port_assets

    ML_portfolio.append(portfolio_daily_returns)
    portfolio_holdings.append((p_start_date, p_end_date, held_tickers))

ML_portfolio_streamed = chain_portfolio_performance(ML_portfolio, starting_value=initial_value)

# Print tickers held at each point
for period_info in portfolio_holdings:
    start, end, tickers = period_info
    print(f"Held from {start} to {end}: {tickers}")

ML_portfolio_streamed

In [ ]:
ML_daily_returns = ML_portfolio_streamed.pct_change()
ML_cumulative_returns = (1 + ML_daily_returns).cumprod()

ML_cumulative_returns.iloc[0] = 1
ML_portfolio_normalized = (ML_cumulative_returns / ML_cumulative_returns.iloc[0]) * 100

Nasdaq_comp = getNasdaq_comp(ML_portfolio_streamed.index[0], ML_portfolio_streamed.index[-1])

fig = go.Figure()

fig.add_trace(go.Scatter(
    x=Nasdaq_comp.index,
    y=Nasdaq_comp['Normalized'],
    mode='lines',
    name='Nasdaq Composite'
))

fig.add_trace(go.Scatter(
    x=ML_cumulative_returns.index,
    y=ML_portfolio_normalized,
    mode='lines',
    name='Portfolio Growth'
))

fig.update_layout(
    title='Comparison of Portfolio vs. Nasdaq Composite Growth',
    xaxis_title='Date',
    yaxis_title='Normalized Value (Base 100)',
    xaxis=dict(
        type='date',
        tickformat='%b %Y',
        tickmode='auto'
    ),
    width=1920,
    height=1080,
    font=dict(
        family="Cambria",
        size=18,
    )
)
fig.show()
#fig.write_html("RBAVsNasdaq.html")
#fig.write_image("RBAVsNasdaq.png", format='png', width=1920, height=1080)

## 7.0 Testing Against Others

In [ ]:
best_port_assets = best_portfolio['tickers']
best_port_assets_test_data = raw_data_test.loc[:, best_port_assets]

Nasdaq_comp = getNasdaq_comp(ML_cumulative_returns.index[0], ML_cumulative_returns.index[-1])

best_portfolio_weights = best_portfolio['weights']
normalized_prices = best_port_assets_test_data.div(best_port_assets_test_data.iloc[0])
daily_returns = normalized_prices.pct_change()
weighted_returns = daily_returns.mul(best_portfolio_weights, axis='columns')
portfolio_daily_returns = weighted_returns.sum(axis=1)
portfolio_cumulative_returns = (1 + portfolio_daily_returns).cumprod()

portfolio_start = portfolio_cumulative_returns.iloc[0]
portfolio_normalized = (portfolio_cumulative_returns / portfolio_start) * 100

fig = go.Figure()

fig.add_trace(go.Scatter(
    x=Nasdaq_comp.index,
    y=Nasdaq_comp['Normalized'],
    mode='lines',
    name='Nasdaq Composite'
))

fig.add_trace(go.Scatter(
    x=ML_cumulative_returns.index,
    y=ML_portfolio_normalized,
    mode='lines',
    name='Portfolio with Adjustments'
))

fig.add_trace(go.Scatter(
    x=ML_cumulative_returns.index,
    y=portfolio_normalized,
    mode='lines',
    name='Base Portfolio'
))

fig.update_layout(
    title='Comparison of Portfolio vs. Nasdaq Composite Growth : 2021-2023',
    xaxis_title='Date',
    yaxis_title='Normalized Value (Base 100)',
    xaxis=dict(
        type='date',
        tickformat='%b %Y',
        tickmode='auto'
    ),
    width=1920,
    height=1080,
    font=dict(
        family="Cambria",
        size=18,
    )
)
fig.show()
fig.write_html("MLRBAvsRBAvsNasdaq.html")
fig.write_image("MLRBAvsRBAvsNasdaq.png", format='png', width=1920, height=1080)

## 8.0 Find Optimal Portfolio Size

In [ ]:
all_portfolios, dominant_portfolios = MonteCarloRBA(names, cov, annualized_returns, 10000, 'sharpe', 3, 50)

In [ ]:
rd_portfolio_sizes = [len(portfolio['tickers']) for portfolio in all_portfolios]
rd_volatility = [np.sqrt(portfolio['variance']) for portfolio in all_portfolios]
rd_returns = [portfolio['return'] for portfolio in all_portfolios]

volatility_by_size = defaultdict(list)
for size, vol, ret in zip(rd_portfolio_sizes, rd_volatility, rd_returns):
    volatility_by_size[size].append((vol, ret))

average_volatility = {size: np.mean([v[0] for v in vols]) for size, vols in volatility_by_size.items()}
average_returns = {size: np.mean([v[1] for v in vols]) for size, vols in volatility_by_size.items()}

sorted_sizes = sorted(average_volatility.keys())
sorted_average_vols = [average_volatility[size] for size in sorted_sizes]
sorted_average_rets = [average_returns[size] for size in sorted_sizes]

fig = go.Figure()
fig.add_trace(go.Scatter(
    x=sorted_sizes,
    y=sorted_average_vols,
    mode='lines',
    name='Average Volatility'
))

fig.update_layout(
    title='Average Volatility and Returns by Portfolio Size',
    xaxis_title='Number of Assets in Portfolio',
    yaxis_title='Average Value',
    xaxis=dict(type='category'),
)

fig.show()

In [ ]:
all_portfolios, dominant_portfolios = MonteCarloRBA(names, cov, annualized_returns, 10000, 'vol', 50, 50)

In [ ]:
rd_portfolio_sizes = [sum(weight > 0 for weight in portfolio['weights']) for portfolio in all_portfolios]

portfolio_size_counts = Counter(rd_portfolio_sizes)

sizes = sorted(portfolio_size_counts.keys())
counts = [portfolio_size_counts[size] for size in sizes]

fig = go.Figure(data=[go.Bar(x=sizes, y=counts)])

fig.update_layout(
    title='Distribution of Portfolio Sizes After Optimization',
    xaxis_title='Number of assets in portfolio after optimizing',
    yaxis_title='Number of Portfolios',
    xaxis=dict(type='category'),
    yaxis=dict(type='linear')
)

fig.show()

In [ ]:
#fig.write_html("PortfolioSize.html")